In [74]:
import sys
import hashlib
if sys.version_info < (3, 6):
    import sha3
from datastructures import St
GD = {}    #giant dictionary

class DLS:
    def __init__(self):
        self.debug= True
        GD  = {}
        pass
    def makedoc(self,xdict,key,nkey):
        skey,snkey = str(int(key)),str(int(nkey))
        hkey = hashlib.sha224(skey+snkey).hexdigest()
        return {"_id":hkey,
                "key":key,
                "nkey":nkey,
                'rvalue':['r\'','r\''], #root value
                "type":'lut',
                "value_ypairs":xdict[key]} #r' is the root of the tree with leaf nodes pairs of y ords.
    
    #return a dictionary object which is a doc in a collection in a db.
    def Get(self,cid,sk): #will get the record, 3 types
        hkey=self.h(cid,sk)
        return dbcollection.find_one({"_id": hkey})

    #it inserts item in mongodb
    def Put(self,key,nkey):
        newdoc=self.makedoc(xdict,key,nkey)
        dbcollection.insert_one(newdoc) #do get doc before
    
    def put(self,key,record):
        #a record can be [pcid,pkey] or structure S or a hash, pcid,pkey = parent collection id, parent key
        GD[key] = record #if mongodb is used, get a record with column "key"=key
        
        #NOTE:
        ''' 
            for a python dictionary, does not matter if record exists for (key,record).
            for mongodb, if (key,record) exists: do-update, else do-insert
        '''
        
    def delete(key):
        try:
            del GD[key]
        except Exceptio,e:
            print(str(e))
        #none
        '''
        try:
            db.collection_c.delete_many({"_id":key}) #which db,collection,
            print '\nDeletion successful\n' 
        except Exception, e:
            print str(e)
    '''
    def get(self,key):  
        #will get the record of 3 differnt types.
        return GD[key]
    
    def h(self,*args):
        s = hashlib.sha3_512()
        for p in args:
            s.update(str(p))
        return s.hexdigest()[0:8]
        
    def Init(self):
        cid = self.h(1,0,0) #id of main collection with parent 0 and key 0
        self.put(cid,[0,0]) #even main collection has an entry
        self.put(self.h(cid,0,0),0) #initialize root of main collection to zero
        print cid
        return cid
        
    def InsertFirstKey(self,cid,k,t):#cid,key,type
        oldhash = self.get(self.h(cid,0,0))
        if oldhash == 0:
            newhash = self.h(k,k,0,t)
            S = St()
            S.key = k
            S.nkey = k;
            S.val = 0;
            S.type_ = t;
            S.x = 0;
            S.y = 0;
            S = S.castToDict()#cast to dict obj
            self.put(self.h(cid,k),S)
            self.put(self.h(cid,0,0), newhash)
        
            [p_cid,p_key] = self.get(cid) #see if cid has parents
            if p_cid != 0: #has parent
                self.Recursive(p_cid,p_key,oldhash,newhash)
            
    def CreateCollection(self,cid,k): #parent cid, parent key
        #a new colleciton is created under any parent collection 'cid', leaf structure S with key 'k'
        Sdic = self.get(self.h(cid,k)) 
        S = St(dic=Sdic) #coz get returns dic type,
        if S.val == 0: #value os zero so we can initialize collection by adding the record
            childcid = self.h(1,cid,k)
            self.put(childcid,[cid,k]) #new collection created
            self.put(self.h(childcid,0,0),0) #initializing root of the childcid hashes.
            return childcid
    
    def InsertKey(self,cid,sk,k):#sk is the key that is split to insert key k
        import copy
        print("InsertKey: cid,sk,k"),cid,sk,k
        Sdic = self.get(self.h(cid,sk)) #cast to S obj
        print("Sdic"), Sdic
        S = St(dic=Sdic)
        SN = copy.deepcopy(S);#new item
        skp = S.nkey
        if not ((k < k < skp) or (k > sk >= skp) or (sk >= skp > k)):
            return #new key k is not enclosed by sk, sk
        S.nkey = k; SN.key = k
        if SN.type_ == 0: 
            SN.val = 0
            
        ox = S.x; 
        oy = S.y #old x, y values of item sk, it's going to be pulled down.
        oldhash_sk = self.get(self.h(cid,ox,oy))
        newhash_sk = self.h(S.key, S.nkey, S.val, SN.type_) #as S.nkey is now different
        hash_k = self.h(SN.key, SN.nkey,SN.val, SN.type_) #hash of new item
        
        #pulling down both items
        S.x = 2*ox; 
        SN.x = 2*ox+1;  
        S.y = SN.y = oy + 1
        
        #write back items
        self.put(self.h(cid,sk),S)
        self.put(self.h(cid,k), SN)
        
        #store new hashes
        self.put(self.h(cid,S.x,S.y),newhash_sk)
        self.put(self.h(cid,SN.x,SN.y),hash_k) 
        
        #compute parent
        newparent = self.h(newhash_sk,hash_k)
        self.put(self.h(cid,ox,oy),newparent) #oldhash_sk was the original value at ox, oy
        [oldroot, newroot] = self.UpdateHashes(cid,ox,oy,oldhash_sk,newparent)
        [p_cid, p_key] = self.get(cid)
        if self.debug: print("Now recursing")
        if p_cid != 0:
            self.Recursive(p_cid,p_key, oldroot,newroot)
            
        
    def UpdateHashes(self,cid,x,y,old,new):
        if self.debug: print("updating hashes...")
        #in collection cid, hash x, y is modified from old to new
        #this functions updates all parent hashes accordingly
        oldroot = self.get(self.h(cid,0,0)) #this is the root of the collection that will be 
        #changed by this function.#along with all hashes on the way to the root
        
        self.put(self.h(cid,x,y),new)#first we will set the new leaf hash;this is duplicate fun.
        
        xc = x;#current values of x and y as we move up the tree.
        yc = y;
        ch = new #current hash. we will keep hash extending with complementary nodes
        
        while yc > 0:
            if (xc % 2 == 1): #xc is odd, the sibiling hash is to the left)
                sh = self.get(self.h(cid,xc-1,yc));#sh is siblihg hash
                ch = self.h(sh,ch) #complementary hash ch, left or right information
            else:
                #sc is even so sibling is to the right
                sh = self.get(self.h(cid,xc+1), yc)
                ch  = self.h(ch,sh)
            #end el
            yc = y-1 #climb up
            xc = xc >> 1 #dividing by 2, move left/right
            
            #@TODO: collect all (sibling hashes) complementary hashes (comp.nodes) for creating certs
            ##also need to add things to collect all complementary hashes to help create certificates
            
            self.put(self.h(cid,xc,yc), ch) #update, this is the new current hash
            
        #endwhile
        #when wile loop is done, the root should be set    
        return oldroot, ch #ch is the new root.
        
    def Recursive(self,cid, key,old, new): #old hash, new hash
        if self.debug:print("recursing to update hashes up the tree.")
        while cid !=0:
            Sdic = self.get(self.h(cid,key)) #coz get returns dic type,
            S = St(dic=Sdic)
            
            oldhash = self.get(self.h(cid, S.x, S.y))
            S.val = new;
            newhash = self.h(S.key,S.nkey,S.val,S.type_)
            self.put(self.h(cid,S.x,S.y),newhash)
            self.UpdateHashes(cid,S.x,S.y,oldhash,newhash)
            [cid,key] = self.get(self.h(1,cid,key))

    def InsertKeyBalanced(cid,sk,k,kb):
        import copy
        #ks is the key that is split
        #k is the inserted key
        #kb is the key pulled down to become sibling of k
        Ssk = self.get(self.h(cid,sk))
        Sk = copy.deepcopy(Ssk)
        Skb = self.get(h(cid,kb))
        
        #First check k enclosed by sk and sk’
        skp = Ssk.nkey
        if not (sk< k< skp) or (k< skp <= sk) or (sk <= skp < k):
            return
        
        Sk.key = k; 
        Ssk.nkey = k; 
        if (Sk.type_== 0):
            Sk.val = 0;
        
        oldhash_ks = self.get(h(cid,Ssk.x,Ssk.y));
        newhash_ks = self.h(Ssk.key,Ssk.nkey,Ssk.val,Ssk.type_);
        
        self.put(h(cid,Ssk.x,Ssk.y),newhash_ks)
        
        [oldroot,newroot] = self.UpdateHashes(cid,Ssk.x,Ssk.y,oldhash_ks,newhash_ks)
        ox = Skb.x; 
        oy = Skb.y; 
        oldhash_kb = self.get(h(cid,ox,oy))
        
        Skb.x = Skb.x<<1;
        Sk.y = Skb.y = Skb.y+1;S
        k.x = Skb.x+1; #make Skb and Sb siblings
        
        hash_k = h(Sk,key.Sk.nkey,Sk.val,Sk.type);
        newparent = self.h(oldhash_kb,hash_k) #this will replace oldhash_
        [newroot,newnewroot] = self.UpdateHashes(cid,ox,oy,oldhash_kb,newparent)
        [p_cid, p_key] = self.get(cid);
        if p_cid != 0:
            self.Recursive(p_cid,p_key, old,newnew);

    def BalanceTree(cid,k1,k2,k3):
        
        #k2 and k3 are siblings. we want to move k2 up one level and make k3 a sibling of k1 
        #(by pulling k1 down one level)
        S1 = self.get(self.h(cid,k1)); 
        S2 = self.get(self.h(cid,k2));
        S3 = get(self.h(cid,k3));
        if (S2.y != S3.y) or  (S3.x-S2.x !=1):
            return;
        oy = S2.y-1;
        ox = S2.x<<1   #ox,oy will be new position of S2
        hashS2S3 = self.get(self.h(cid,ox,oy)); #get parent hash of S2S3 will need to change it to hash of S2
        hashS2 = self.get(self.h(cid,S2.x,S2.y)); #value to be assigned to ox,oy
        
        self.put(self.h(cid,ox,oy),hashS2); 
        self.put(self.h(cid,k2),S2); #and done
        
        self.delete(self.h(cid,ox>>1,oy+1));
        self.delete(self.h(cid,ox>>1+1,oy+1));#old S2 and S3
        [old, new] = self.UpdateHashes(cid,ox,oy,hashS2S3,hashS2);

        hashS3 = self.get(self.h(cid,S3.x,S3.y));
        hashS1 = self.get(self.h(cid,S1.x,S1.y)); 
        hashS1S3 = self.h(hashS1,hashS3) 
        
        ox = S1.x; 
        oy = S1.y #this is the position that will change from hashS1 -> hashS1S3
        
        S1.x = ox>>1  ; 
        S3.x = ox>>1+1; 
        S1.y = S3.y=oy+1 #//change x and y in records
        
        self.put(h(cid,k1),S1);  
        self.put(h(cid,k3),S3); #//write back modified records
        
        #now write back new hashes
        self.put(h(cid,S1.x,S1.y),hashS1); 
        self.put(h(cid,S3.x,S3.y),hashS3);
        self.put(h(cid,oox,ooy),hashS1S3);
        
        [new,newnew] = self.UpdateHashes(cid, ox,oy, hashS1, hashS1S3) ;
        [p_cid, p_key] = self.get(cid);
        if p_cid != 0:
            self.Recursive(p_cid,p_key, old,newnew)
    
    def SwapLeaves(cid,k1,k2):#Swap k1 and k2
        
        S1 = self.get(self.h(cid,k1)); 
        S2 = self.get(self.h(cid,k2));
        h1 = self.get(self.h(cid,S1.x,S1.y));
        h2 = self.get(self.h(cid,S2.x,S2.y));
        y1 = S1.y;
        x1 = S1.x;

        S1.x = S2.x;
        S1.y = S2.y;
        S2.x = x1;
        S2.y = y1; #swap positions
        self.put(self.h(cid,k1),S1); 
        self.put(self.h(cid,k2), S2);
        self.put(self.h(cid,S1.x,S1.y), h1); 
        
        [old,new] = self.UpdateHashes(cid,S1.x,S1.y,h2,h1);
        self.put(self.h(cid,S2.x,S2.y), h2);
        [new,newnew] = self.UpdateHashes(cid,S2.x,S2.y,h1,h2);
        [p_cid, p_key] = self.get(cid);
        if p_cid != 0:
            self.Recursive(p_cid,p_key, old,newnew);
    
    def GetComplementaryNodes(x,y):
        '''for an entry structure S with x and y position values, 
        it returns complementary nodes of a hash-node @ position x, y in in hash-tree at '''
        compnodes = []
        cury = y #current y
        curx = x
        while cury > 0:
            if curx%2 == 1:
                #left side is sibling
                compnodes +=[self.get(self.h(cid,curx-1, cury))]
            else:
                compnones +=[self.get(self.h(cid,curx+1, cury))]
            cury = cury-1
            curx = curx/2
            
        return compnodes

In [47]:
from pymongo import MongoClient
import datetime
import hashlib
import dlsbuilder as DLSB
reload(DLSB)

client = MongoClient("mongodb://localhost:27017")
db = client.dlsdb #database 
print client.database_names() #list all the databses in the server.
mapgrids= db.mapgrids #collection x1-x2:[(y1,y2)...] i.e rectangles at each x-ord
dbcollections=mapgrids #setting global dbcollection

[u'admin', u'dlsdb', u'local', u'test', u'test2']


In [75]:
#Drop collection if exists.
db.drop_collection('mapgrids')

#document
import DataSources as mydatasource
import sys
import collections
reload(mydatasource)

datatype = 'sample'
polygons, inputsspatialref,feature_names = mydatasource.getPolygons(datatype=datatype,dofilter=True)
print("len of features_names"), len(polygons)
xdict,n,tn=DLSB.workpolygon(polygons) #for complete map with npolygons
print xdict
keys = sorted(xdict.keys())


dlsObj = DLS()
grids=[]
for i in range(len(keys)-1):
    key,nkey = keys[i],keys[i+1]
    grids.append(dlsObj.makedoc(xdict,key,nkey))
    
key,nkey = keys[-1], keys[0] #last wrap up.
grids.append(dlsObj.makedoc(xdict,key,nkey))    
mapgridid=mapgrids.insert_many(grids)

##
cid = dlsObj.Init()
keys = xdict.keys()
pkey1 = keys[0] #select any first key in parent colleciton
t = 0 #0-> dict, 1-> LUT
dlsObj.InsertFirstKey(cid,pkey1,1)
childcid = dlsObj.CreateCollection(cid, pkey1) #parent is cid, key is pkey1
print GD
print("childcid"), cid, pkey1,childcid
dlsObj.InsertFirstKey(childcid,xdict[pkey1][0][0], 1) #it puts value of key to 0, see S.val=0 in function. !!!!
cur_ckey = xdict[pkey1][0][0] #current child key, tuple item
for tup in xdict[pkey1][1:]: #all other tuples
    dlsObj.InsertKey(childcid,cur_ckey,tup[0])
    cur_ckey = tup[0]

curr_key = firstKey
for newkey in keys[1:]:
    dlsObj.InsertKey(cid,curr_key,newkey)
    curr_key = newkey
print("InsertKey"),GD

len of features_names 3
{1.0: [(0.0, 1.0), (1.0, 3.0), (3.0, 5.0)], 2.0: [(4.0, 5.0), (5.0, 6.0), (0.0, 1.0)], 3.0: [(4.0, 4.0), (6.0, 8.0), (1.0, 1.0)], 4.0: [(3.0, 4.0), (8.0, 9.0), (1.0, 3.0)], 5.0: [(3.0, 3.0), (9.0, 10.0), (2.0, 3.0)], 6.0: [(3.0, 3.0), (10.0, 10.0), (1.0, 2.0)], 7.0: [(3.0, 4.0), (10.0, 10.0), (1.0, 1.0)], 10.0: [(4.0, 4.0), (9.0, 10.0), (1.0, 2.0)], 11.0: [(2.0, 4.0), (2.0, 2.0), (7.0, 9.0), (2.0, 4.0)], 13.0: [(2.0, 7.0)]}
3a3d69a6
{'46a80644': '3b15c7cc', '7888e1c0': {'key': 1.0, 'val': 0, 'nkey': 1.0, 'type_': 1, 'posx': 0, 'posy': 0}, '3a3d69a6': [0, 0], 'ba3ae81c': 0, '802674d8': ['3a3d69a6', 1.0]}
childcid 3a3d69a6 1.0 802674d8
recursing to update hashes up the tree.


AttributeError: St instance has no attribute 'k'

In [4]:
db.collection_names(include_system_collections=False) #list all the collections in a database

[u'mapgrids', u'test']

(3, 4)